FONTE: https://www.gov.br/secretariageral/pt-br/acesso-a-informacao/informacoes-classificadas-e-desclassificadas
VIDEO ETL: https://www.youtube.com/watch?v=c-0SF0WVW7s&t=3s

In [22]:
pip install --upgrade google-auth

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install pandas-gbq -U

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     -------------------------------------- 215.1/215.1 kB 4.4 MB/s eta 0:00:00
  Using cached google_auth_oauthlib-0.8.0-py2.py3-none-any.whl (19 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
     --------------------------------------- 20.3/20.3 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 187.7/187.7 kB ? eta 0:00:00
     ------------------------------------- 527.0/527.0 kB 34.5 MB/s eta 0:00:00
  Using cached googleapis_common_protos-1.58.0-py2.py3-none-any.whl (223 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     ---------------------------------------- 77.4/77.4 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 39.7 MB/s eta 0:00:00
     ---------------------------------------- 47.9/47.9 kB ? eta 0:00:00
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)


In [10]:
# lib
import pandas as pd
from google.oauth2 import service_account

In [11]:
# read csv
df = pd.read_csv('data_cartao.csv', sep=',', encoding='latin-1')

# drops and regex
df = df.drop(df.columns[[3,7]], axis=1).drop([113340, 113341]) 
df['cpf_cnpj'] = df['cpf_cnpj'].str.replace(".", "", regex=True).replace(",", "", regex=True).fillna(0)
df['valor'] = df['valor'].replace('[R&&$]', "", regex=True).replace('[.]', "", regex=True).replace(',', ".", regex=True).replace('[ &&-]', "0", regex=True)
df

,data,cpf_cnpj,empresa,valor,tipo,categoria
0,02/01/2003,31349202000177,JW TRANSPORTADORA TUROSTICA,1696.90,D,LOCACAO DE MEIOS DE TRANSPORTE
1,02/01/2003,31349202000177,JW TRANSPORTADORA TUROSTICA,1336.60,D,LOCACAO DE MEIOS DE TRANSPORTE
2,02/01/2003,31349202000177,JW TRANSPORTADORA TUROSTICA,1396.43,D,LOCACAO DE MEIOS DE TRANSPORTE
3,03/01/2003,592717000170,FLORES ALVORADA,8585.00,C,OUTROS SERVIEOS DE TERCEIROS-PESSOA JURODICA
4,03/01/2003,839308000125,CRISTAL LIMPEZA - COMERCIO DE MATERIAIS PARA L...,251.50,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAECO
...,...,...,...,...,...,...
113335,03/12/2022,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,179.5,C,FORNECIMENTO DE ALIMENTAECO
113336,03/12/2022,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,54.95,C,GENEROS DE ALIMENTAECO
113337,04/12/2022,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIECO,518.07,C,GENEROS DE ALIMENTAECO
113338,04/12/2022,12550441000146,RESTAURANTE SOHO,330.52,C,FORNECIMENTO DE ALIMENTAECO


In [12]:
# count para validar se todos os registros estão compatíveis 
df.count()

data         113340
cpf_cnpj     113340
empresa      113340
valor        113340
tipo         113340
categoria    113340
dtype: int64

In [13]:
# classificação dos dados
df.data = pd.to_datetime(df.data, format = '%d/%m/%Y').dt.date
df['cpf_cnpj'] = df['cpf_cnpj'].astype('int64')
df['valor'] = df['valor'].astype(float)
df['empresa'] = df['empresa'].astype(pd.StringDtype())
df['tipo'] = df['tipo'].astype(pd.StringDtype())
df['categoria'] = df['categoria'].astype(pd.StringDtype())
df

,data,cpf_cnpj,empresa,valor,tipo,categoria
0,2003-01-02,31349202000177,JW TRANSPORTADORA TUROSTICA,1696.90,D,LOCACAO DE MEIOS DE TRANSPORTE
1,2003-01-02,31349202000177,JW TRANSPORTADORA TUROSTICA,1336.60,D,LOCACAO DE MEIOS DE TRANSPORTE
2,2003-01-02,31349202000177,JW TRANSPORTADORA TUROSTICA,1396.43,D,LOCACAO DE MEIOS DE TRANSPORTE
3,2003-01-03,592717000170,FLORES ALVORADA,8585.00,C,OUTROS SERVIEOS DE TERCEIROS-PESSOA JURODICA
4,2003-01-03,839308000125,CRISTAL LIMPEZA - COMERCIO DE MATERIAIS PARA L...,251.50,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAECO
...,...,...,...,...,...,...
113335,2022-12-03,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,179.50,C,FORNECIMENTO DE ALIMENTAECO
113336,2022-12-03,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,54.95,C,GENEROS DE ALIMENTAECO
113337,2022-12-04,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIECO,518.07,C,GENEROS DE ALIMENTAECO
113338,2022-12-04,12550441000146,RESTAURANTE SOHO,330.52,C,FORNECIMENTO DE ALIMENTAECO


In [14]:
# confirmação dos types
df.dtypes

data          object
cpf_cnpj       int64
empresa       string
valor        float64
tipo          string
categoria     string
dtype: object

In [15]:
credentials = service_account.Credentials.from_service_account_file(filename='GBQ.json', 
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [23]:
df.to_gbq(credentials=credentials, 
          destination_table='cartao_corporativo.cartao_csv', 
          if_exists='replace', 
          table_schema=[{'name': 'data', 'type': 'DATE'}])

100%|██████████| 1/1 [00:00<?, ?it/s]
